## Web Crawing
- 웹 페이지에서 데이터를 수집하는 방법에 대해서 학습

### 웹크롤링 방법

#### 웹페이지의 종류
- 정적인 페이지 : 웹 브라우져에 화면이 한번 뜨면 이벤트에 의한 화면의 변경이 없는 페이지 
- 동적인 페이지 : 웹 브라우져에 화면이 뜨고 이벤트가 발생하면 서버에서 데이터를 가져와 화면을 변경하는 페이지

#### requests 이용
- 받아오는 문자열에 따라 두가지 방법으로 구분
    - json 문자열로 받아서 파싱하는 방법 : 주로 동적 페이지 크롤링할때 사용 
    - html 문자열로 받아서 파싱하는 방법 : 주로 정적 페이지 크롤링할때 사용
        
#### selenium 이용
- 브라우져를 직접 열어서 데이터를 받는 방법

#### 크롤링 방법에 따른 속도
- requests json > requests html > selenium

### Crwaling Naver Stock Datas
- 네이버 증권 사이트에서 주가 데이터 수집
- 수집할 데이터 : 일별 kospi, kosdaq 주가, 일별 환율(exchange rate) 데이터
- 데이터 수집 절차
    - 웹서비스 분석 : url
    - 서버에 데이터 요청 : request(url) > response : json(str)
    - 서버에서 받은 데이터 파싱(데이터 형태를 변경) : json(str) > list, dict > DataFrame

In [1]:
# 필요한 패키지 임포트

import warnings
warnings.filterwarnings('ignore') # 경고 문구 없애기

import pandas as pd
import requests



#### 1. 웹서비스 분석 : url
- pc 웹페이지가 복잡하면 mobile 웹페이지에서 수집

In [3]:
# 어떤 url에서 가져와야 좋을지 이 과정에서 크롬 개발자 도구를 사용할 것

# page, page_size = 1, 10
# url = 'https://m.stock.naver.com/api/index/KOSPI/price?pageSize={page_size}&page={page}'
# url

page, page_size = 1, 10 
url = f'https://m.stock.naver.com/api/index/KOSPI/price?pageSize={page_size}&page={page}' 
url

'https://m.stock.naver.com/api/index/KOSPI/price?pageSize=10&page=1'

#### 2. 서버에 데이터 요청 : request(url) > response : json(str)
- response의 status code가 200이 나오는지 확인
- 403이나 500이 나오면 request가 잘못되거나 web server에서 수집이 안되도록 설정이 된것임
    - header 설정 또는 selenium 사용
- 200이 나오더라도 response 안에 있는 내용을 확인 > 확인하는 방법 : response.text

In [5]:
# JSON 포맷의 문자열 데이터로 받아올 것
# 요청한다! 그리고 보내주는 것을 받아오는 게 리스폰스 객체

response = requests.get(url)
response

# 200이 정상적으로 받아왔다는 상태 코드임

<Response [200]>

In [8]:
# 요청해서 받아오고, 이 문자열이 리스폰스 안에 들어있는데 어떤 게 들어있는지 확인하고 싶으면

# response.text
response.text[:300] # 지금은 dtype이 string
# 그래서 df로 바꾸기 위해 파싱 과정을 거쳐야 함

'[{"localTradedAt":"2023-02-16","closePrice":"2,473.46","compareToPreviousClosePrice":"45.56","compareToPreviousPrice":{"code":"2","text":"상승","name":"RISING"},"fluctuationsRatio":"1.88","openPrice":"2,444.06","highPrice":"2,474.62","lowPrice":"2,442.07"},{"localTradedAt":"2023-02-15","closePrice":"2'

#### 3. 서버에서 받은 데이터 파싱(데이터 형태를 변경) : json(str) > list, dict > DataFrame

In [9]:
# 일단 타입 확인
type(response)






requests.models.Response

In [11]:
# 리스폰스 클래스 안에는 다양한 좋은 함수들이 있음
data = response.json() # 위에 문자열을 제이슨이란 메서드를 활용해서 리스트로 바꿔줌
data # 리스트 안에 딕셔너리가 있는 포맷임... 딕셔너리 데이터 하나 = row 데이터 

[{'localTradedAt': '2023-02-16',
  'closePrice': '2,473.46',
  'compareToPreviousClosePrice': '45.56',
  'compareToPreviousPrice': {'code': '2', 'text': '상승', 'name': 'RISING'},
  'fluctuationsRatio': '1.88',
  'openPrice': '2,444.06',
  'highPrice': '2,474.62',
  'lowPrice': '2,442.07'},
 {'localTradedAt': '2023-02-15',
  'closePrice': '2,427.90',
  'compareToPreviousClosePrice': '-37.74',
  'compareToPreviousPrice': {'code': '5', 'text': '하락', 'name': 'FALLING'},
  'fluctuationsRatio': '-1.53',
  'openPrice': '2,473.09',
  'highPrice': '2,473.44',
  'lowPrice': '2,424.16'},
 {'localTradedAt': '2023-02-14',
  'closePrice': '2,465.64',
  'compareToPreviousClosePrice': '12.94',
  'compareToPreviousPrice': {'code': '2', 'text': '상승', 'name': 'RISING'},
  'fluctuationsRatio': '0.53',
  'openPrice': '2,471.33',
  'highPrice': '2,480.34',
  'lowPrice': '2,459.83'},
 {'localTradedAt': '2023-02-13',
  'closePrice': '2,452.70',
  'compareToPreviousClosePrice': '-17.03',
  'compareToPreviousPri

In [12]:
# 딕셔너리 -> df 
df = pd.DataFrame(data)
df

,localTradedAt,closePrice,compareToPreviousClosePrice,compareToPreviousPrice,fluctuationsRatio,openPrice,highPrice,lowPrice
0,2023-02-16,"2,473.46",45.56,"{'code': '2', 'text': '상승', 'name': 'RISING'}",1.88,"2,444.06","2,474.62","2,442.07"
1,2023-02-15,"2,427.90",-37.74,"{'code': '5', 'text': '하락', 'name': 'FALLING'}",-1.53,"2,473.09","2,473.44","2,424.16"
2,2023-02-14,"2,465.64",12.94,"{'code': '2', 'text': '상승', 'name': 'RISING'}",0.53,"2,471.33","2,480.34","2,459.83"
3,2023-02-13,"2,452.70",-17.03,"{'code': '5', 'text': '하락', 'name': 'FALLING'}",-0.69,"2,461.96","2,462.58","2,440.86"
4,2023-02-10,"2,469.73",-11.79,"{'code': '5', 'text': '하락', 'name': 'FALLING'}",-0.48,"2,470.65","2,473.80","2,454.74"
5,2023-02-09,"2,481.52",-2.12,"{'code': '5', 'text': '하락', 'name': 'FALLING'}",-0.09,"2,470.66","2,487.14","2,466.61"
6,2023-02-08,"2,483.64",31.93,"{'code': '2', 'text': '상승', 'name': 'RISING'}",1.30,"2,469.42","2,487.35","2,465.96"
7,2023-02-07,"2,451.71",13.52,"{'code': '2', 'text': '상승', 'name': 'RISING'}",0.55,"2,440.14","2,456.17","2,432.40"
8,2023-02-06,"2,438.19",-42.21,"{'code': '5', 'text': '하락', 'name': 'FALLING'}",-1.70,"2,461.82","2,469.42","2,438.04"
9,2023-02-03,"2,480.40",11.52,"{'code': '2', 'text': '상승', 'name': 'RISING'}",0.47,"2,466.54","2,485.08","2,460.21"


In [13]:
# 원하는 것만 가져오기
df = pd.DataFrame(data)[['localTradedAt', 'closePrice']]
df

,localTradedAt,closePrice
0,2023-02-16,"2,473.46"
1,2023-02-15,"2,427.90"
2,2023-02-14,"2,465.64"
3,2023-02-13,"2,452.70"
4,2023-02-10,"2,469.73"
5,2023-02-09,"2,481.52"
6,2023-02-08,"2,483.64"
7,2023-02-07,"2,451.71"
8,2023-02-06,"2,438.19"
9,2023-02-03,"2,480.40"


#### 4. 함수로 만들기

In [14]:
# def stock_price(code, page, page_size):
def stock_price(code = 'KOSPI', page = 1, page_size = 20): # default 설정 naver는 page 수 60개 제한 있음 지금
    ''' This function is crawling stock price form naver. # 함수 설명
    params: # 파라미터
        code : str : KOSPI, KOSDAQ
        page : int
        page_size : int
    return: # 함수를 실행했을 때 나오는 데이터 타입
        type : DataFrame
    ''' # 독스트링
    # 1. URL
    url = f'https://m.stock.naver.com/api/index/{code}/price?pageSize={page_size}&page={page}' 

    # 2. request(URL) > response : 받아지는 것은 json(str)
    response = requests.get(url)
    
    # 3. json(str) > list, dict > DataFrame
    data = response.json()
    # 두개의 컬럼만 선택해서 리턴
    return pd.DataFrame(data)[['localTradedAt', 'closePrice']]


In [16]:
# 1번째 페이지의 20개의 데이터를 가져오고 싶다
df = stock_price('KOSDAQ', 1, 20)
df

,localTradedAt,closePrice
0,2023-02-16,783.32
1,2023-02-15,765.46
2,2023-02-14,779.58
3,2023-02-13,772.55
4,2023-02-10,772.44
5,2023-02-09,784.58
6,2023-02-08,779.98
7,2023-02-07,772.79
8,2023-02-06,761.33
9,2023-02-03,766.79


#### 5. 원달러 환율 데이터 수집 : 실습

#### 6. 시각화

#### 7. 데이터 스케일링
- min max scaling


- $z = \frac{x_i - min(x)}{max(x) - min(x)} (0 \leqq z \leqq 1)$


- latex syntax : `https://jjycjnmath.tistory.com/117`

#### 8. 상관관계 분석
- 피어슨 상관계수(Pearson Correlation Coefficient)
- 두 데이터 집합의 상관도를 분석할때 사용되는 지표
- 상관계수의 해석
    - -1에 가까울수록 서로 반대방향으로 움직임
    - 1에 가까울수록 서로 같은방향으로 움직임
    - 0에 가까울수록 두 데이터는 관계가 없음